# Convert parquet file to postgres table

In [ ]:
# get driver to postgres
#!spark-shell --packages org.postgresql:postgresql:42.2.18 

In [1]:
import pyspark
from pyspark import SparkConf
from pyspark.sql import SparkSession
import psycopg2

In [2]:
spark = SparkSession \
    .builder \
    .appName("spark_to_postgres") \
    .getOrCreate()

In [3]:
locations = spark.read.parquet("OUT_DATA/nyt_locations_geography/")

In [4]:
locations.printSchema()

root
 |-- fips: string (nullable = true)
 |-- county: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- location_id: long (nullable = true)
 |-- state: string (nullable = true)



In [5]:
locations.count()

3272

In [5]:
# create and write table nyt_locations_geography in postgres
locations.write\
    .format("jdbc")\
    .option("url", "jdbc:postgresql:capstone")\
    .option("dbtable", "nyt_locations_geography")\
    .option("user","postgres")\
    .option("password", "postgres")\
    .save()